In [1]:
import pandas as pd
from datetime import datetime
import numpy as np

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/history.csv")
columns_to_remove = ['id', 'country_code', 'currency_code', 'net_of_fees', 'cashflows_custody_fee', 'fee_paid_separately', 'custody_fee_withdrawal',
                     'is_fee_exempt', 'include_client_consolidation', 'credit_limit_type', 'sss_type', 'sss_agent', 'is_midwest_clearing_account',
                     'terminal_code', 'target_grantor_grantee_flag', 'iso_funds_code', 'esir_number', 'shareholder_language', 'shareholder_language',
                     'conjunction', 'function_code', 'tms_settlement_location', 'portfolio_cost_method', 'portfolio_name_address_option',
                     'portfolio_report_option', 'portfolio_summary_option', 'interactive_portfolio_code', 'deceased_fair_market_value', 'rep_commission_rate',
                     'dup_trip_quad_code', 'special_fee_code', 'non_calendar_year_end', 'resp_specimen_plan', 'sss_location', 'last_maintenance_user', 'last_maintenance_time',
                     'retail_last_maintenance_user', 'retail_last_maintenance_time', 'arp_pension_origin']

cols_to_numerify_from_bool = ["is_registered", "is_active",
    "use_client_address", "is_spousal", "is_arp_locked",
    "use_hand_delivery", "use_mail", "share_name_address_to_issuer",
    "shareholder_instructions_received", "rrsp_limit_reached", "is_portfolio_account",
    "has_no_min_commission", "is_tms_eligible", "is_agent_bbs_participant",
    "is_parameters_account", "is_spousal_transfer", "spousal_age_flag", "has_multiple_name",
    "discretionary_trading_authorized", "receive_general_mailings", "has_discrete_auth",
    "is_non_objecting_beneficial_owner", "is_objecting_to_disclose_info", "consent_to_pay_for_mail",
    "consent_to_email_delivery", "has_received_instruction", "is_broker_account",
    "is_inventory_account", "is_gl_account", "is_control_account", "is_extract_eligible",
    "is_pledged", "is_resp", "use_original_date_for_payment_calc", "is_family_resp",
    "is_hrdc_resp", "is_plan_grandfathered", "is_olob", "visible_in_reports", 'inserted_at', 'updated_at']

cols_to_encode = ['class_id', 'type_code', 'debit_code', 'contract_type',
                  'branch', 'retail_plan',
                  'special_tag', 'guarantee_gtor_type', 'dividend_confirm_code', 'options_trading_type',
                  'interest_dividend_conversion_type']

cols_null_to_numeric = ['rep_commission_override', 'loan_limit_override', 'non_plan_book_value_flag']

cols_date_to_numeric = ['last_trade_date', 'inception_date', 'last_update_date', 'plan_effective_date',
                        'plan_end_date', 'rrif_original_date']

special_cols = ['language_code', 'title', 'risk_tolerance', 'investment_objective', 'mailing_consent', 'number_of_beneficiaries', 'label']

# drop useless columns
data = data.drop(columns=columns_to_remove)

# store for check later
data_columns = data.columns

data.columns

In [ ]:
# get an overview of what the dataset looks like
data.head()

In [ ]:
import matplotlib.pyplot as plt
counts = data['type_code'].value_counts()

# Combine elements with less than 2% occurrence into an 'other' category
total_count = counts.sum()
counts = counts[counts / total_count >= 0.02]
other_count = total_count - counts.sum()
counts['Other'] = other_count

# Create a pie chart
plt.figure(figsize=(8, 8))
plt.pie(counts, labels=counts.index, autopct='%1.1f%%', startangle=90)
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle
plt.title('Unique Account Types Offered')
plt.show()

In [ ]:
# bool to numeric function (cols_to_numerify_from_bool)
def bool_to_numeric(df, feature):
    bool_to_numeric_dict = {"t": 1, "f": 0}
    df[feature] = df[feature].map(bool_to_numeric_dict).fillna(-1)
    return df

for col in cols_to_numerify_from_bool:
  try:
    data = bool_to_numeric(data, col)
  except Exception as e:
    print(f'Error: {e}')
    continue

data[cols_to_numerify_from_bool].head()

In [ ]:
# one hot encode function (cols_to_encode)
def encode_and_bind(original_dataframe, feature_to_encode):
    temp = pd.get_dummies(original_dataframe[[feature_to_encode]])
    df = pd.concat([original_dataframe, temp], axis=1)
    df = df.drop([feature_to_encode], axis=1)
    return(df)

for col in cols_to_encode:
  try:
    data = encode_and_bind(data, col)
  except Exception as e:
    print(f'Error: {e}')
    continue

data.head()

In [ ]:
# for these, null = 0, has value = 1 (cols_null_to_numeric)
def null_to_numeric(df, feature):
  df[feature] = df[feature].notnull().astype(int)
  return df

for col in cols_null_to_numeric:
  try:
    data = null_to_numeric(data, col)
  except Exception as e:
    print(f'Error: {e}')
    continue

data[cols_null_to_numeric].head()

In [ ]:
# modifying (cols_date_to_numeric)
date_format = "%Y-%m-%d"

# Assuming `data` is your DataFrame and `date_columns` is a list of date columns
for column in cols_date_to_numeric:
    try:
        # Convert to datetime
        data[column] = pd.to_datetime(data[column], errors='coerce')

        # Calculate the difference in days from today
        data[column] = (datetime.now() - data[column]).dt.days

    except Exception as e:
        print(f'Error: {e}')
        continue

# Calculate the average of non-null values for each column
column_averages = {column: data[column].dropna().mean() for column in cols_date_to_numeric}

# Replace null values with the average of non-null values
for column in cols_date_to_numeric:
    data[column] = data[column].fillna(column_averages[column])

data[cols_date_to_numeric].head()

In [ ]:
# SPECIAL COLUMN: language_code
# english = 0 , french = 1
def update_language(df, feature):
  language_to_numeric_dict = {"F": 1, "E": 0}
  df[feature] = df[feature].map(language_to_numeric_dict)
  return df

data = update_language(data, 'language_code')

data['language_code'].head()

In [ ]:
# SPECIAL COLUMN: title
def gender_to_number(df, feature):
    mapping = {
        'male': ['mr', 'monsieur', 'sir', 'mister'],
        'female': ['miss', 'ms', 'mrs', 'madame', 'mme', 'mlle', 'mademoiselle', 'mm']
    }
    for key in mapping.keys():
        mapping[key] = [phrase.lower() for phrase in mapping[key]]

    def map_gender(value):
        try:
            value_lower = str(value).lower()
            if pd.isna(value) or value == "":
                return "none_provided"
            elif any(phrase in value_lower for phrase in mapping['male']):
                return "male"
            elif any(phrase in value_lower for phrase in mapping['female']):
                return "female"
            else:
                return "other"
        except AttributeError:
            return "none_provided"
    df[feature] = df[feature].apply(map_gender)
    df = encode_and_bind(df, feature)
    return df

data = gender_to_number(data, 'title')

In [ ]:
# SPECIAL COLUMN: risk_tolerance, investment_objective
def convert_risk_to_numeric(df, feature):

    unique_risks = set()

    for idx, row in df.iterrows():
        risk_str = row[feature]
        risk_str = str(risk_str)
        if pd.notna(risk_str):
            for char in risk_str:
                if char.isalpha():
                    unique_risks.add(char)

    col_names = [f'{feature}_{risk}' for risk in unique_risks]

    for col_name in col_names:
        df[col_name] = 0

    df[f'{feature}_exists'] = 1

    for idx, row in df.iterrows():
        risk_str = row[feature]
        risk_str = str(risk_str)
        if pd.notna(risk_str):
            for i in range(len(risk_str)):
                if risk_str[i].isalpha():
                    col_name = f'{feature}_{risk_str[i]}'
                    j = i + 1
                    while j < len(risk_str) and risk_str[j].isdigit():
                        j += 1
                    try:
                        value = int(risk_str[i + 1:j]) if j > i + 1 else 0
                    except (ValueError, IndexError):
                        value = 0
                        df.at[idx, f'{feature}_exists'] = 0
                    df.at[idx, col_name] = value
    df = df.fillna(0)
    df.drop(columns=[feature], inplace=True)

    return df

data = convert_risk_to_numeric(data, 'risk_tolerance')
data = convert_risk_to_numeric(data, 'investment_objective')

data.head()

In [ ]:
# SPECIAL COLUMNS: mailing_consent, number_of_beneficiaries, label
data['mailing_consent'] = data['mailing_consent'].fillna(0)
data['number_of_beneficiaries'] = data['number_of_beneficiaries'].fillna(0)
data['label'] = data['label'].map({'Churn': 1, 'No Churn': 0})

# fill all other null cells with -1
data = data.fillna(-1)

In [ ]:
# THE FINAL DATASET
data.head()

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.cross_decomposition import PLSRegression
from sklearn.feature_selection import mutual_info_regression
from xgboost import XGBClassifier

NUMBER_OF_FEATURES_PLS = 50
NUMBER_OF_FEATURES_MRMR = 30

train_f1 = []
test_f1 = []

# SPLITTING UP THE DATA INTO INPUT AND OUTPUT
X = data.drop(columns=['label'])
y = data['label']

# SCALE THE DATA
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# IMPLEMENT PLS REGRESSION FOR DIMENSIONALITY REDUCTION
pls = PLSRegression(n_components=NUMBER_OF_FEATURES_PLS)
pls.fit(X_scaled, y)
X_pls = pls.transform(X_scaled)

# IMPLEMENTING MRMR
# Feature Relevance
mi_scores = mutual_info_regression(X_pls, y)

# Feature Redundancy
cos_sim = np.abs(np.corrcoef(X_pls.T))

# Calculate MRMR
mrmr_scores = mi_scores / np.maximum(np.max(mi_scores) - np.mean(mi_scores), 1e-10)
mrmr_scores -= np.mean(cos_sim, axis=1)

# Select Features using MRMR
selected_features = np.argsort(mrmr_scores)[::-1][:NUMBER_OF_FEATURES_MRMR]


selected_feature_names = X.columns[selected_features]
print(selected_feature_names)
data = data[selected_feature_names]
data['label'] = y
data.head()

In [ ]:
data.columns

In [ ]:
# we need to determine the optimal amount of clusters
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# Assuming data is a NumPy array or a pandas DataFrame

# Elbow method to determine the optimal number of clusters
sse = []
silhouette_scores = []
k_range = range(2, 15)
for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(data)
    sse.append(kmeans.inertia_)
    silhouette_scores.append(silhouette_score(data, kmeans.labels_))

# Plotting Elbow Method
plt.figure(figsize=(10, 5))
plt.plot(k_range, sse, 'bo-')
plt.xlabel('Number of clusters (k)')
plt.ylabel('Sum of Squared Distances (SSE)')
plt.title('Elbow Method For Optimal k')
plt.show()

# Plotting Silhouette Scores
plt.figure(figsize=(10, 5))
plt.plot(k_range, silhouette_scores, 'bo-')
plt.xlabel('Number of clusters (k)')
plt.ylabel('Silhouette Score')
plt.title('Silhouette Scores For Different k')
plt.show()